# CASE STUDY 10
William Nayden and Quynh Chau
March 15, 2021

## INTRODUCTION

The purpose of this case study is to use Russian Housing data to explore how different assumptions for data imputation
affect predictive model performance. We use XGBoost with parameter tuning as an example. 

## METHOD FOR MISSING DATA IMPUTATION

We have 24,449 rows of data that are missing data in some column. We want to test the effect of removing and imputing the missing data.

Our baseline models were performed using a data set of 6,042 rows, with the missing data removed.

For numerical variables with missing values, the median values of each variable were used for data imputation.  The median was chose to minimize any outliers and skewness in the data.  
    
For the categorical variables with missing values, frequent categorical data imputation was done using the mode for each column.

## METHOD FOR MODELING

We decided to use XGBoost for this model since it performed well in previous case studies.

Our `Baseline` model is a non-tuned XGBoost model trained on both our imputed and non-imputed data sets.

Our `XGBoost` model is a parameter tuned XGBoost model trained on both our imputed and non-imputed data sets.

# RESULTS

On our `Baseline` model, imputation had no effect on the RMSE.

However, imputation drastically reduced RMSE for our `XGBoost` model, which already outperformed our `Baseline` model based on RMSE.

|Model|Imputed Data|Non-Imputed Data|
|--- -|------------|----------------|
Baseline|6,636,678|6,636,678|
XGBoost|2,518,834|4,007,743|

## CONCLUSION

Based on RMSE evaluation criteria, the `XGBoost` model performed better with imputed data than with non-imputed data given 
the same baseline calibration and parameter tuning. We recommend performing permutation for this data set, and tuning the parameters of `XGBoost` to maximize predictive ability.

# CODE APPENDIX

In [1]:
import pandas as pd
import numpy as np
import os as os

## Reading in Russian Housing Data from local directory

In [2]:
os.getcwd()

'C:\\Users\\Owner'

In [3]:
os.chdir('C:/SMU/DS 7333 Quant the World/Case Studies/Case Study week 10')

In [4]:
df = pd.read_csv("RussianHousing.csv")

In [227]:
df3 = df.copy()

In [228]:
print("The data set has {} rows and {} columns".format(df.shape[0], df.shape[1]))
display(df.describe())
display(df.head())
display(df.dtypes.value_counts())

The data set has 30471 rows and 292 columns


,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
count,30471.000000,30471.000000,24088.000000,30304.000000,20899.000000,20899.000000,1.686600e+04,20899.000000,20899.000000,16912.000000,...,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,3.047100e+04
mean,15237.917397,54.214269,34.403271,7.670803,12.558974,1.827121,3.068057e+03,1.909804,6.399301,2.107025,...,32.058318,10.783860,1.771783,15.045552,30.251518,0.442421,8.648814,52.796593,5.987070,7.123035e+06
std,8796.501536,38.031487,52.285733,5.319989,6.756550,1.481154,1.543878e+05,0.851805,28.265979,0.880148,...,73.465611,28.385679,5.418807,29.118668,47.347938,0.609269,20.580741,46.292660,4.889219,4.780111e+06
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+05
25%,7620.500000,38.000000,20.000000,3.000000,9.000000,1.000000,1.967000e+03,1.000000,1.000000,1.000000,...,2.000000,1.000000,0.000000,2.000000,9.000000,0.000000,0.000000,11.000000,1.000000,4.740002e+06
50%,15238.000000,49.000000,30.000000,6.500000,12.000000,1.000000,1.979000e+03,2.000000,6.000000,2.000000,...,8.000000,2.000000,0.000000,7.000000,16.000000,0.000000,2.000000,48.000000,5.000000,6.274411e+06
75%,22855.500000,63.000000,43.000000,11.000000,17.000000,2.000000,2.005000e+03,2.000000,9.000000,3.000000,...,21.000000,5.000000,1.000000,12.000000,28.000000,1.000000,7.000000,76.000000,10.000000,8.300000e+06
max,30473.000000,5326.000000,7478.000000,77.000000,117.000000,6.000000,2.005201e+07,19.000000,2014.000000,33.000000,...,377.000000,147.000000,30.000000,151.000000,250.000000,2.000000,106.000000,218.000000,21.000000,1.111111e+08


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,8/20/2011,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,8/23/2011,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,8/27/2011,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,9/1/2011,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,9/5/2011,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


int64      157
float64    119
object      16
dtype: int64

## Separating categorical and numerical columns in the data frame

In [229]:
num_vars = df3.columns[df.dtypes !='object']
cat_vars = df3.columns[df.dtypes == 'object']

In [230]:
print(num_vars)
print(cat_vars)

Index(['id', 'full_sq', 'life_sq', 'floor', 'max_floor', 'material',
       'build_year', 'num_room', 'kitch_sq', 'state',
       ...
       'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
       'cafe_count_5000_price_high', 'big_church_count_5000',
       'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
       'sport_count_5000', 'market_count_5000', 'price_doc'],
      dtype='object', length=276)
Index(['timestamp', 'product_type', 'sub_area', 'culture_objects_top_25',
       'thermal_power_plant_raion', 'incineration_raion',
       'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion',
       'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion',
       'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology'],
      dtype='object')


# Impute Missing Numerical Data with the Median Value

In [231]:
df3.fillna(df.median())

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,8/20/2011,43,27.0,4.0,12.0,1.0,1979.0,2.0,6.0,...,9,4,0,13,22,1,0,52,4,5850000
1,2,8/23/2011,34,19.0,3.0,12.0,1.0,1979.0,2.0,6.0,...,15,3,0,15,29,1,10,66,14,6000000
2,3,8/27/2011,43,29.0,2.0,12.0,1.0,1979.0,2.0,6.0,...,10,3,0,11,27,0,4,67,10,5700000
3,4,9/1/2011,89,50.0,9.0,12.0,1.0,1979.0,2.0,6.0,...,11,2,1,4,4,0,0,26,3,13100000
4,5,9/5/2011,77,77.0,4.0,12.0,1.0,1979.0,2.0,6.0,...,319,108,17,135,236,2,91,195,14,16331452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,30469,6/30/2015,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,...,15,5,0,15,26,1,2,84,6,7400000
30467,30470,6/30/2015,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,...,313,128,24,98,182,1,82,171,15,25000000
30468,30471,6/30/2015,45,30.0,10.0,20.0,1.0,1979.0,1.0,1.0,...,1,1,0,2,12,0,1,11,1,6970959
30469,30472,6/30/2015,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,...,22,1,1,6,31,1,4,65,7,13500000


# Impute Missing Categorical Variables with Most Frequent Values

In [232]:
df3['timestamp']= df3['timestamp'].fillna(df3['timestamp'].mode()[0])
df3['product_type'] = df3['product_type'].fillna(df3['product_type'].mode()[0])

In [233]:
df3['sub_area']= df3['sub_area'].fillna(df3['sub_area'].mode()[0])
df3['culture_objects_top_25']= df3['culture_objects_top_25'].fillna(df3['culture_objects_top_25'].mode()[0])

In [234]:
df3['thermal_power_plant_raion']= df3['thermal_power_plant_raion'].fillna(df3['thermal_power_plant_raion'].mode()[0])
df3['incineration_raion']=df3['incineration_raion'].fillna(df3['incineration_raion'].mode()[0])

In [235]:
df3['oil_chemistry_raion'] = df3['oil_chemistry_raion'].fillna(df3['oil_chemistry_raion'].mode()[0])
df3['radiation_raion']=df3['radiation_raion'].fillna(df3['radiation_raion'].mode()[0])

In [236]:
df3['railroad_terminal_raion']= df3['railroad_terminal_raion'].fillna(df3['railroad_terminal_raion'].mode()[0])
df3['big_market_raion'] = df3['big_market_raion'].fillna(df3['big_market_raion'].mode()[0])

In [237]:
df3['nuclear_reactor_raion'] = df3['nuclear_reactor_raion'].fillna(df3['nuclear_reactor_raion'].mode()[0])
df3['detention_facility_raion'] = df3['detention_facility_raion'].fillna(df3['detention_facility_raion'].mode()[0])

In [238]:

df3['big_road1_1line'] = df3['big_road1_1line'].fillna(df3['big_road1_1line'].mode()[0])

## Label Encoding Imputed Categorical Data

In [239]:
Y_3 = df3['price_doc']

In [240]:
df3.drop('price_doc',inplace = True, axis=1)

In [241]:
print (Y_3)

0         5850000
1         6000000
2         5700000
3        13100000
4        16331452
           ...   
30466     7400000
30467    25000000
30468     6970959
30469    13500000
30470     5600000
Name: price_doc, Length: 30471, dtype: int64


In [242]:
from sklearn.preprocessing import LabelEncoder

cols_3 = ['timestamp', 'product_type', 'sub_area', 'culture_objects_top_25','thermal_power_plant_raion','incineration_raion',
       'oil_chemistry_raion', 'radiation_raion','railroad_terminal_raion','big_market_raion','nuclear_reactor_raion',
       'detention_facility_raion','water_1line','big_road1_1line','railroad_1line','ecology']

df3[cols_3]= df3[cols_3].apply(LabelEncoder().fit_transform)

In [243]:
x_3 = df3
print(x_3)

          id  timestamp  full_sq  life_sq  floor  max_floor  material  \
0          1       1013       43     27.0    4.0        NaN       NaN   
1          2       1023       34     19.0    3.0        NaN       NaN   
2          3       1033       43     29.0    2.0        NaN       NaN   
3          4       1066       89     50.0    9.0        NaN       NaN   
4          5       1145       77     77.0    4.0        NaN       NaN   
...      ...        ...      ...      ...    ...        ...       ...   
30466  30469        879       44     27.0    7.0        9.0       1.0   
30467  30470        879       86     59.0    3.0        9.0       2.0   
30468  30471        879       45      NaN   10.0       20.0       1.0   
30469  30472        879       64     32.0    5.0       15.0       1.0   
30470  30473        879       43     28.0    1.0        9.0       1.0   

       build_year  num_room  kitch_sq  ...  cafe_count_5000_price_1500  \
0             NaN       NaN       NaN  ...       

In [244]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.33
X_train3, X_test3, y_train3, y_test3 = train_test_split(x_3, Y_3, test_size=test_size, random_state=seed)

In [245]:
import xgboost as xgb
dtrain3 = xgb.DMatrix(X_train3, label = y_train3)
dtest3 = xgb.DMatrix(X_test3, label = y_test3)

C:\Users\Owner\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [246]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

# "Learn" the mean from the training data
mean_train3 = np.mean(y_train3)
# Get predictions on the test set
baseline_predictions3 = np.ones(y_test3.shape) * mean_train3
# Compute MAE
rmse_baseline3 = sqrt(mean_squared_error(y_test3, baseline_predictions3))
print("Imputed Data Baseline RMSE is {:.2f}".format(rmse_baseline))

Imputed Data Baseline RMSE is 6636678.71


In [247]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [248]:
params['eval_metric'] = "rmse"

In [249]:
num_boost_round = 999

In [250]:
model3 = xgb.train(
    params,
    dtrain3,
    num_boost_round=num_boost_round,
    evals=[(dtest3, "Test")],
    early_stopping_rounds=10
)

print("Best RMSE Using Imputed Data: {:.2f} with {} rounds".format(
                 model3.best_score,
                 model3.best_iteration+1))

[18:01:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-rmse:6.18403e+06
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:4.73193e+06
[2]	Test-rmse:3.85791e+06
[3]	Test-rmse:3.31457e+06
[4]	Test-rmse:2.98876e+06
[5]	Test-rmse:2.81134e+06
[6]	Test-rmse:2.72388e+06
[7]	Test-rmse:2.66383e+06
[8]	Test-rmse:2.63229e+06
[9]	Test-rmse:2.6124e+06
[10]	Test-rmse:2.5967e+06
[11]	Test-rmse:2.5878e+06
[12]	Test-rmse:2.58198e+06
[13]	Test-rmse:2.57698e+06
[14]	Test-rmse:2.56997e+06
[15]	Test-rmse:2.57049e+06
[16]	Test-rmse:2.558e+06
[17]	Test-rmse:2.55252e+06
[18]	Test-rmse:2.54112e+06
[19]	Test-rmse:2.53938e+06
[20]	Test-rmse:2.53445e+06
[21]	Test-rmse:2.53586e+06
[22]	Test-rmse:2.53292e+06
[23]	Test-rmse:2.53418e+06
[24]	Test-rmse:2.53237e+06
[25]	Test-rmse:2.53077e+06
[26]	Test-rmse:2.53239e+06
[27]	Test-rmse:2.52896e+06
[28]	Test-rmse:2.52826e+06
[29]	Test-rmse:2.52723e+06
[30]	Test-rmse:2.52535e+06
[31]	

# XGBoost Not Using Data Imputation

In [119]:
df2 = df.copy() 

print("Before removing rows with missing values")
print(df2.shape)
print("After removing rows with missing values")
df2.dropna(inplace=True)
print(df2.shape)

Before removing rows with missing values
(30471, 292)
After removing rows with missing values
(6042, 292)


In [120]:
df2.dtypes.value_counts()

int64      157
float64    119
object      16
dtype: int64

In [121]:
Y = df2['price_doc']

In [ ]:
df2.drop('price_doc',inplace = True, axis=1)

In [125]:
print (Y)

8056      2750000
8154      7100000
8287     11700000
8387     10400000
8391      6200000
           ...   
30462    10544070
30463    12000000
30467    25000000
30469    13500000
30470     5600000
Name: price_doc, Length: 6042, dtype: int64


In [126]:
from sklearn.preprocessing import LabelEncoder

cols = ['timestamp', 'product_type', 'sub_area', 'culture_objects_top_25','thermal_power_plant_raion','incineration_raion',
       'oil_chemistry_raion', 'radiation_raion','railroad_terminal_raion','big_market_raion','nuclear_reactor_raion',
       'detention_facility_raion','water_1line','big_road1_1line','railroad_1line','ecology']

le = LabelEncoder()

df2[cols]= df2[cols].apply(LabelEncoder().fit_transform)

In [127]:
x = df2
print(x)

          id  timestamp  full_sq  life_sq  floor  max_floor  material  \
8056    8059        365       11     11.0    2.0        5.0       2.0   
8154    8157        376       45     27.0    6.0        9.0       1.0   
8287    8290        444       77     50.0    3.0        5.0       2.0   
8387    8390        455       56     29.0    5.0       16.0       5.0   
8391    8394        455       31     21.0    5.0        9.0       5.0   
...      ...        ...      ...      ...    ...        ...       ...   
30462  30465        448       47     30.0   23.0       25.0       4.0   
30463  30466        448       56     29.0   13.0       14.0       1.0   
30467  30470        448       86     59.0    3.0        9.0       2.0   
30469  30472        448       64     32.0    5.0       15.0       1.0   
30470  30473        448       43     28.0    1.0        9.0       1.0   

       build_year  num_room  kitch_sq  ...  cafe_count_5000_price_1500  \
8056       1907.0       1.0      12.0  ...       

In [128]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=test_size, random_state=seed)

In [139]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

C:\Users\Owner\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [157]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

# "Learn" the mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
rmse_baseline = sqrt(mean_squared_error(y_test, baseline_predictions))
print("Baseline RMSE is {:.2f}".format(rmse_baseline))

Baseline RMSE is 6636678.71


In [158]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [159]:
params['eval_metric'] = "rmse"

In [160]:
num_boost_round = 999

In [161]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best RMSE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

[16:30:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-rmse:8.41435e+06
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:6.70424e+06
[2]	Test-rmse:5.63442e+06
[3]	Test-rmse:4.94776e+06
[4]	Test-rmse:4.54288e+06
[5]	Test-rmse:4.31495e+06
[6]	Test-rmse:4.20954e+06
[7]	Test-rmse:4.15261e+06
[8]	Test-rmse:4.11282e+06
[9]	Test-rmse:4.08169e+06
[10]	Test-rmse:4.06497e+06
[11]	Test-rmse:4.02831e+06
[12]	Test-rmse:4.02163e+06
[13]	Test-rmse:4.00967e+06
[14]	Test-rmse:4.00963e+06
[15]	Test-rmse:4.00936e+06
[16]	Test-rmse:4.00774e+06
[17]	Test-rmse:4.02303e+06
[18]	Test-rmse:4.02656e+06
[19]	Test-rmse:4.02722e+06
[20]	Test-rmse:4.0296e+06
[21]	Test-rmse:4.03231e+06
[22]	Test-rmse:4.02901e+06
[23]	Test-rmse:4.01382e+06
[24]	Test-rmse:4.01689e+06
[25]	Test-rmse:4.0204e+06
[26]	Test-rmse:4.02354e+06
Stopping. Best iteration:
[16]	Test-rmse:4.00774e+06

Best RMSE: 4007743.75 with 17 rounds


In [164]:
print (x)

          id  timestamp  full_sq  life_sq  floor  max_floor  material  \
8056    8059        365       11     11.0    2.0        5.0       2.0   
8154    8157        376       45     27.0    6.0        9.0       1.0   
8287    8290        444       77     50.0    3.0        5.0       2.0   
8387    8390        455       56     29.0    5.0       16.0       5.0   
8391    8394        455       31     21.0    5.0        9.0       5.0   
...      ...        ...      ...      ...    ...        ...       ...   
30462  30465        448       47     30.0   23.0       25.0       4.0   
30463  30466        448       56     29.0   13.0       14.0       1.0   
30467  30470        448       86     59.0    3.0        9.0       2.0   
30469  30472        448       64     32.0    5.0       15.0       1.0   
30470  30473        448       43     28.0    1.0        9.0       1.0   

       build_year  num_room  kitch_sq  ...  cafe_count_5000_price_1500  \
8056       1907.0       1.0      12.0  ...       

In [165]:
missingcols = df.columns[df.isna().any()].tolist()
print(missingcols)

['life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'preschool_quota', 'school_quota', 'hospital_beds_raion', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_monolith', 'build_count_panel', 'build_count_foam', 'build_count_slag', 'build_count_mix', 'raion_build_count_with_builddate_info', 'build_count_before_1920', 'build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995', 'build_count_after_1995', 'metro_min_walk', 'metro_km_walk', 'railroad_station_walk_km', 'railroad_station_walk_min', 'ID_railroad_station_walk', 'cafe_sum_500_min_price_avg', 'cafe_sum_500_max_price_avg', 'cafe_avg_price_500', 'cafe_sum_1000_min_price_avg', 'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000', 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_200

In [166]:
missingvals = df.isnull().sum()
missingvals[missingvals >0]

life_sq                                   6383
floor                                      167
max_floor                                 9572
material                                  9572
build_year                               13605
num_room                                  9572
kitch_sq                                  9572
state                                    13559
preschool_quota                           6688
school_quota                              6685
hospital_beds_raion                      14441
raion_build_count_with_material_info      4991
build_count_block                         4991
build_count_wood                          4991
build_count_frame                         4991
build_count_brick                         4991
build_count_monolith                      4991
build_count_panel                         4991
build_count_foam                          4991
build_count_slag                          4991
build_count_mix                           4991
raion_build_c

In [167]:
missingdf = df[missingcols]
print(missingdf)

       life_sq  floor  max_floor  material  build_year  num_room  kitch_sq  \
0         27.0    4.0        NaN       NaN         NaN       NaN       NaN   
1         19.0    3.0        NaN       NaN         NaN       NaN       NaN   
2         29.0    2.0        NaN       NaN         NaN       NaN       NaN   
3         50.0    9.0        NaN       NaN         NaN       NaN       NaN   
4         77.0    4.0        NaN       NaN         NaN       NaN       NaN   
...        ...    ...        ...       ...         ...       ...       ...   
30466     27.0    7.0        9.0       1.0      1975.0       2.0       6.0   
30467     59.0    3.0        9.0       2.0      1935.0       4.0      10.0   
30468      NaN   10.0       20.0       1.0         NaN       1.0       1.0   
30469     32.0    5.0       15.0       1.0      2003.0       2.0      11.0   
30470     28.0    1.0        9.0       1.0      1968.0       2.0       6.0   

       state  preschool_quota  school_quota  ...  cafe_sum_2000

In [168]:
datatypes = missingdf.dtypes
print(datatypes)

life_sq                                  float64
floor                                    float64
max_floor                                float64
material                                 float64
build_year                               float64
num_room                                 float64
kitch_sq                                 float64
state                                    float64
preschool_quota                          float64
school_quota                             float64
hospital_beds_raion                      float64
raion_build_count_with_material_info     float64
build_count_block                        float64
build_count_wood                         float64
build_count_frame                        float64
build_count_brick                        float64
build_count_monolith                     float64
build_count_panel                        float64
build_count_foam                         float64
build_count_slag                         float64
build_count_mix     

In [169]:
missingdf['state']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
30466    3.0
30467    3.0
30468    1.0
30469    2.0
30470    2.0
Name: state, Length: 30471, dtype: float64

In [170]:
g = df.columns.to_series().groupby(df.dtypes).groups
g

{dtype('int64'): Index(['id', 'full_sq', 'raion_popul', 'children_preschool',
        'preschool_education_centers_raion', 'children_school',
        'school_education_centers_raion',
        'school_education_centers_top_20_raion', 'healthcare_centers_raion',
        'university_top_20_raion',
        ...
        'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
        'cafe_count_5000_price_high', 'big_church_count_5000',
        'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
        'sport_count_5000', 'market_count_5000', 'price_doc'],
       dtype='object', length=157),
 dtype('float64'): Index(['life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room',
        'kitch_sq', 'state', 'area_m', 'green_zone_part',
        ...
        'green_part_3000', 'prom_part_3000', 'cafe_sum_3000_min_price_avg',
        'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'green_part_5000',
        'prom_part_5000', 'cafe_sum_5000_min_price_avg',
        '

In [152]:
import matplotlib.pyplot as plt
# Define the number of rows and columns you want
#n_rows=30471
#n_cols=51
# Create the subplots
#fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols)

#for i, column in enumerate(missingdf.columns):
   # sns.distplot(missingdf[column],ax=axes[i//n_cols,i%n_cols])